In [ ]:
!pip install datasets==2.10.1
#load_dataset sometimes hangs on a higher version
!pip install transformers

# 1. Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
data_path = ""

# 2. Connect to Github

In [48]:
!git config --global user.name "hpeter11"
!git config --global user.email "hilton_petersen@brown.edu"

In [49]:
!git clone https://github.com/hpeter11/BERT_QA_System.git

fatal: destination path 'BERT_QA_System' already exists and is not an empty directory.


In [52]:
!git pull origin main

From https://github.com/hpeter11/BERT_QA_System
 * branch            main       -> FETCH_HEAD
Auto-merging BERT_QA.ipynb
CONFLICT (content): Merge conflict in BERT_QA.ipynb
Automatic merge failed; fix conflicts and then commit the result.


In [55]:
!mv /content/drive/MyDrive/BERT_QA.ipynb /content/BERT_QA_System/

mv: cannot stat '/content/drive/MyDrive/BERT_QA.ipynb': No such file or directory


In [ ]:
%cd /content/BERT_QA_System/

/content/BERT_QA_System


In [ ]:
from google.colab import userdata
key = userdata.get('git_key')

In [ ]:
# Test
!git remote set-url origin https://{key}@github.com/hpeter11/BERT_QA_System.git
!git add BERT_QA.ipynb
!git commit -m "Periodic update from Colab"
!git push

fatal: pathspec 'BERT_QA.ipynb' did not match any files
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
To https://github.com/hpeter11/BERT_QA_System.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/hpeter11/BERT_QA_System.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


# Preprocessing

In [ ]:
from datasets import load_dataset

import torch
import numpy as np
import random

# we set up some seeds so that we can reproduce results
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


In [ ]:
"""
Some options for BERT model that can be run in colab:

"distilbert-base-uncased",
"distilbert-base-uncased-distilled-squad",
"distilbert-base-cased",
"distilbert-base-cased-distilled-squad",

"""

In [ ]:
# Change train.json / dev.json to the appropriate filepaths =====
data_files = {"train": "train.json", "dev": "dev.json"}
dataset = load_dataset('json', data_files=data_files)

In [ ]:
def main():
  '''Here's the basic structure of the main block -- feel free to add or
  remove parameters/helper functions as you see fit, but all steps here are
  needed and we expect to see precision, recall, and f1 scores printed out'''
  device = "cuda" if torch.cuda.is_available() else "cpu"
  batch_size = 64

  model, tokenizer = load_model()
  train, validation = load_data()

  train_data_loader = preprocess_and_tokenize(train)
  validation_data_loader = preprocess_and_tokenize(validation)

  train_losses, val_losses = train_loop(train_data_loader, validation_data_loader)
  precision, recall, f1_score  = eval_loop(validation_data_loader)

  print("PRECISION: ", precision)
  print("RECALL: ", recall)
  print("F1-SCORE: ", f1_score)

if __name__ == "__main__":
  main()